# Lectura y preprocesamiento de datos

In [1]:
import pandas as pd
import datetime
import numpy as np
from IPython.display import clear_output

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_excel("data.xlsb")
data["Fecha_Reporte"] = data["Fecha_Reporte"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Ocurrencia"] = data["Fecha_Ocurrencia"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Pago"] = data["Fecha_Pago"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))

In [3]:
aux = data.groupby(["Fecha_Pago"])["Importe USD"].sum().reset_index()

In [4]:
fecha_inicial = aux["Fecha_Pago"].min()
fecha_final = aux["Fecha_Pago"].max()

rango_fechas = pd.date_range(fecha_inicial, fecha_final, freq="D")

In [5]:
#Si una fecha no tiene registro, se agrega con importe 0
aux = aux.set_index("Fecha_Pago")
aux = aux.reindex(rango_fechas, fill_value = 0)
aux = aux.reset_index(names = ["Fecha_Pago", "Importe USD"])

In [6]:
X = aux[["Fecha_Pago", "Importe USD"]]
y = aux[["Importe USD"]]

In [7]:
X["anno"] = X["Fecha_Pago"].apply(lambda x : x.year)
X["Mes"] = X["Fecha_Pago"].apply(lambda x : x.month)
X["Dia"] = X["Fecha_Pago"].apply(lambda x : x.day)
X["Dia_Semana"] = X["Fecha_Pago"].apply(lambda x : x.weekday())

In [8]:
#Ahora cada uno de estos valores los transformaremos a un encoding cíclico.
X["Dia_Semana_sin"] = X["Dia_Semana"].apply(lambda x : np.sin(2*np.pi*x/7))
X["Dia_Semana_cos"] = X["Dia_Semana"].apply(lambda x : np.cos(2*np.pi*x/7))

In [9]:
X["Mes_sin"] = X["Mes"].apply(lambda x : np.sin(2*np.pi*x/12))
X["Mes_cos"] = X["Mes"].apply(lambda x : np.cos(2*np.pi*x/12))

In [10]:
#Los días dependerán del mes, pues hay meses que tienen 30 días y otros 31, además de febrero que tiene 28 o 29.
X["Dia_sin"] = X.apply(lambda x : np.sin(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)
X["Dia_cos"] = X.apply(lambda x : np.cos(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)

In [11]:
#El año lo estandarizamos, pero comenzando desde 0.
X["anno"] = (X["anno"] - X["anno"].min()) / (X["anno"].max() - X["anno"].min())

In [12]:
columns = ["anno", "Dia_Semana_sin", "Dia_Semana_cos", "Mes_sin", "Mes_cos", "Dia_sin", "Dia_cos", "Importe USD"]

In [13]:
#Normalizamos los valores de y
y_mean = y.values.mean()
y_std = y.values.std()
y = (y - y_mean) / y_std
X["Importe USD"] = (X["Importe USD"] - y_mean) / y_std

In [14]:
#Creamos el X_train e y_train utilizando secuencias de un largo fijo
sequence_length = 30
X_train = []
y_train = []

for i in range(sequence_length, len(X)):
    X_train.append(X.iloc[i - sequence_length : i][columns].values)
    y_train.append(y.iloc[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [15]:
X_train.shape, y_train.shape

((5071, 30, 8), (5071, 1))

# Implementación del modelo

In [16]:
n_steps = 30
n_features = 8

In [17]:
def create_model(n_lstm, sequence_length = 30):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(n_lstm, input_shape=(n_steps, n_features)))
    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = tf.keras.losses.MeanSquaredError(),
    )

    return model

In [18]:
best_model = None
best_loss = float("inf")
loss = float("inf")

sizes = [8, 16, 32, 64, 128]

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model(sizes[i % len(sizes)])
    model.fit(X_train, y_train, epochs = 1000, batch_size = 32, callbacks = [tf.keras.callbacks.EarlyStopping("loss", patience = 10, restore_best_weights = True)])

    loss = model.evaluate(X_train, y_train, verbose = 0)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: 0.15655617415905
Iteration 16 - Loss: 0.5968172550201416
Epoch 1/1000
159/159 [==============================] - 3s 7ms/step - loss: 1.0099
Epoch 2/1000
159/159 [==============================] - 1s 6ms/step - loss: 1.0036
Epoch 3/1000
159/159 [==============================] - 1s 7ms/step - loss: 1.0025
Epoch 4/1000
159/159 [==============================] - 1s 7ms/step - loss: 1.0008
Epoch 5/1000
159/159 [==============================] - 1s 7ms/step - loss: 1.0001
Epoch 6/1000
159/159 [==============================] - 1s 8ms/step - loss: 0.9987
Epoch 7/1000
159/159 [==============================] - 1s 7ms/step - loss: 0.9982
Epoch 8/1000
159/159 [==============================] - 1s 8ms/step - loss: 0.9987
Epoch 9/1000
159/159 [==============================] - 1s 7ms/step - loss: 0.9971
Epoch 10/1000
159/159 [==============================] - 1s 7ms/step - loss: 0.9960
Epoch 11/1000
159/159 [==============================] - 1s 7ms/step - loss: 0.9953
Epoch 12/1000
159

In [ ]:
best_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 64)                18688     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 18,753
Trainable params: 18,753
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model.save("model_lstm_64.h5")

INFO:tensorflow:Assets written to: ram://9998df7e-4c31-4766-b2cf-62ce4b01bb15/assets


INFO:tensorflow:Assets written to: ram://9998df7e-4c31-4766-b2cf-62ce4b01bb15/assets
